In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import matplotlib.pyplot as plt
import krakenex
from pykrakenapi import KrakenAPI
import datetime as dt
from datetime import datetime, timezone
import pytz
import string
import time

import sqlite3
conn = sqlite3.connect('<your_SQLlite_db>')


api = krakenex.API(key='<your_key>',secret='<your_secret>')
k = KrakenAPI(api)

#symbol = 'ETH'
#symbol = 'BTC'
symbol = 'ADA'
#symbol = 'ALGO'

base = 'USDT'
base_alt = 'USDT'
pricebase = 'USD'

avail_limit = 10

time.sleep(20)

In [4]:
fiveago = dt.datetime.now(pytz.timezone('US/Pacific')) - dt.timedelta(minutes=0)
print(symbol+" - "+str(fiveago))
wallet = k.get_account_balance()
print(wallet)
op_active = False

ADA - 2022-02-15 12:17:17.545606-08:00
                   vol
USD.HOLD  0.000000e+00
ADA       0.000000e+00
XETH      9.000000e-09
ALGO      0.000000e+00
USDT      1.656346e+02


In [5]:
ret = conn.execute('''SELECT rowid,* from active_operations where active = 1 and symbol = 'ADA' order by timestamp desc limit 1;''')
for row in ret:
    print('ID: ',row[0])
    print('TS: ',row[1])
    print('SY: ',row[2])
    print('PR: ',row[3])
    print('BS: ',row[4])
    op_active = True
if op_active == False:
    print("No Active operations")

No Active operations


In [6]:
if (op_active and round(dt.datetime.now().timestamp()) - row[1] > 900):
    ticker = k.get_ticker_information(symbol+base)
    print(ticker)
    sell_price = str(max(float(ticker['b'][0][0]),float(float(ticker['a'][0][0]))))
    ord_vol = str(float(wallet.loc[symbol,'vol']))
    print(str(ord_vol))
    my_order = k.add_standard_order(symbol+base, type='sell',ordertype='limit',volume=ord_vol,price=sell_price,validate=False)
    print(my_order)
    act_op_query = string.Template("UPDATE active_operations SET active = 0 WHERE rowid = $myid;")
    generate_query = act_op_query.substitute(myid=row[0])
    print(generate_query)
    conn.execute(generate_query)
    conn.commit()
